In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from credentials import FacebookCookies

driver_manager = ChromeDriverManager(latest_release_url="https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.60/linux64/chrome-linux64.zip").install()
driver_service = Service(driver_manager)
driver_options = webdriver.ChromeOptions()https://web.facebook.com/chonlonglay/posts/pfbid02LXyjFmChaSwNCS7DaMYatr8KzbnMM5yQX7LCJuGXbDQRFw9JmunNR5LKzP3sjYMXl
cookies_mngr = FacebookCookies()

chrome = webdriver.Chrome(service=driver_service, options=driver_options)
chrome.get("https://mbasic.facebook.com")
for cookie in cookies_mngr.load():
    chrome.add_cookie(cookie)

# https://web.facebook.com/chonlonglay/posts/pfbid02LXyjFmChaSwNCS7DaMYatr8KzbnMM5yQX7LCJuGXbDQRFw9JmunNR5LKzP3sjYMXl
chrome.get("https://www.facebook.com/colinkkhong/posts/pfbid0EJpdnmCAtXuiD3QhMomVfMUZoqgqkFABYhC7P8QRPAUt1rPm3EC42XaM92X6W8j4l")
orig_tab = chrome.current_window_handle

In [44]:
def show_all_replies(driver, threshold):
    cnt = 1
    while cnt <= threshold:
        all_replied_comments = driver.find_elements(By.XPATH, "//div[contains(@class, 'x1n2onr6 x46jau6')]")
        if len(all_replied_comments) == 0:
            break

        for comment in all_replied_comments:
            cnt += 1
            driver.execute_script("window.scrollBy(0, -50);")  # Cuộn lên 50 dòng
            try:
                view_more_buttons = WebDriverWait(comment, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'x78zum5 x1iyjqo2 x21xpn4 x1n2onr6')]")))
                view_more_buttons.click()
                for sub_cmt in comment:
                # driver.execute_script("window.scrollTo(0, 900);")
                    try:
                        view_more_sub_buttons = WebDriverWait(sub_cmt, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'x78zum5 x1iyjqo2 x21xpn4 x1n2onr6')]")))
                        view_more_sub_buttons.click()
                    except:
                        break
            except Exception as e:
                break

In [45]:
import re

def parse_text(text_element: WebElement):
    text = text_element.get_attribute("innerHTML")
    text = re.sub(r"(<img[^>]*alt=\"([^\"]+)\")[^>]*>", r"\2", text)
    text = re.sub(r"<a[^>]*>(.*?)</a>", r"href(\1)", text)
    text = re.sub(r"(?<=</div>)()(?=<div)", r"\n", text)
    text = re.sub(r"<.*?>", "", text)
    return text

In [46]:
def extract_cmt_attachment_type(cmt_div: WebElement):
        # Sticker: img, xz74otr x1uzojwf x10e4vud xa4qsjk xoj058f x1nxgg22 x10l6tqk x17qophe x13vifvy
        # Image: img, xz74otr
        # Video: video, x1lliihq x5yr21d xh8yej3
        # GIF: div, x1i10hfl x1ypdohk xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz xjyslct xjbqb8w x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x5muytz x1lliihq x5yr21d xdj266r x11i5rnm xat24cr x1mh8g0r x6ikm8r x10wlt62 xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 x16tdsg8 xh8yej3 x1ja2u2z
        # External link: a, x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1ey2m1c xds687c x10l6tqk x17qophe x13vifvy xi2jdih

        content_divs = cmt_div.find_elements(By.XPATH, "div")
        if len(content_divs) == 2:
            return "no attachment"
        
        attm_div = content_divs[1]
        try:
            attm_div.find_element(By.XPATH, ".//img[contains(@class, 'xz74otr x1uzojwf x10e4vud xa4qsjk xoj058f x1nxgg22 x10l6tqk x17qophe x13vifvy')]")
            return "sticker"
        except: pass

        try:
            attm_div.find_element(By.XPATH, ".//img[contains(@class, 'xz74otr')]")
            return "image"
        except: pass

        try:
            attm_div.find_element(By.XPATH, ".//video[contains(@class, 'x1lliihq x5yr21d xh8yej3')]")
            return "video"
        except: pass

        try:
            attm_div.find_element(By.XPATH, ".//div[contains(@class, 'x1i10hfl x1ypdohk xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz xjyslct xjbqb8w x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x5muytz x1lliihq x5yr21d xdj266r x11i5rnm xat24cr x1mh8g0r x6ikm8r x10wlt62 xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 x16tdsg8 xh8yej3 x1ja2u2z')]")
            return "gif"
        except: pass

        try:
            attm_div.find_element(By.XPATH, ".//video[contains(@class, 'x1lliihq x5yr21d xh8yej3')]")
            return "video"
        except: pass

        return "unknown"

In [47]:
chrome.find_element(By.XPATH, "//div[@class='x9f619 x1n2onr6 x1ja2u2z xt0psk2 xuxw1ft']").click()
chrome.find_element(By.XPATH, "//div[@class='x4k7w5x x1h91t0o x1beo9mf xaigb6o x12ejxvf x3igimt xarpa2k xedcshv x1lytzrv x1t2pt76 x7ja8zs x1n2onr6 x1qrby5j x1jfb8zj']").find_element(By.XPATH, "(//div[@role='menuitem'])[3]").click()
show_all_replies(chrome, 50)

In [48]:
data = []
comments = chrome.find_elements(By.XPATH, "//div[contains(@class, 'x1r8uery x1iyjqo2 x6ikm8r x10wlt62 x1pi30zi')]")
for i, comment in enumerate(comments):
    attachment_type = extract_cmt_attachment_type(comment)
    if attachment_type == "image":
        try:
            text_div = comment.find_element(By.XPATH, ".//div[contains(@class, 'xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs')]")
        except: continue
        # img = comment.find_element(By.XPATH, "(div)[2]").find_element(By.XPATH, "//img[contains(@class, 'xz74otr')]")
        img = comment.find_element(By.XPATH, "(div)[2]").find_element(By.TAG_NAME, "img")
        
        text = parse_text(text_div)
        img_src = img.get_attribute("src")

        data.append({
            "text": text,
            "image": img_src
        })

In [49]:
data

[{'text': 'href(Tô Duy Hoàn)',
  'image': 'https://scontent.fsgn2-10.fna.fbcdn.net/v/t39.30808-6/445720346_122196903722008466_4302767549527818087_n.jpg?stp=dst-jpg_s261x260&_nc_cat=109&ccb=1-7&_nc_sid=5f2048&_nc_eui2=AeFQVqol38JQTpdV28gGeUfThkJAz5ZNFY6GQkDPlk0VjhtK_p2tdA_7bBusYJkWoK7b3_ROZiwj58Zbu_Q-I2S4&_nc_ohc=yCT_LviFOQkQ7kNvgFPdaF0&_nc_ht=scontent.fsgn2-10.fna&cb_e2o_trans=q&oh=00_AYDsiULzYWO0BhEQxSBCnvQrVSTiTBfqBdA3GILDAH3vLA&oe=665567A1'},
 {'text': 'href(Tô Duy Hoàn) SONE-218 khỏi cảm ơn',
  'image': 'https://scontent.fsgn2-9.fna.fbcdn.net/v/t39.30808-6/442468955_2814997375323837_970758398908167654_n.jpg?stp=dst-jpg_p75x225&_nc_cat=106&ccb=1-7&_nc_sid=5f2048&_nc_eui2=AeHzAOlHGV3ZPIs-MRpYJVaU2ddEdu-rAJHZ10R276sAkTbmW2Ki_uVGWj88Ioq5CamAsKmNAVuns0aiKmxq-97U&_nc_ohc=O8DyfAPWar0Q7kNvgHWzvEn&_nc_ht=scontent.fsgn2-9.fna&cb_e2o_trans=q&oh=00_AYAuqtdakev71KW0abwuRLyWmk2UVgaMhdI--hbnqPQW5w&oe=665563E8'},
 {'text': 'href(Hocsinh3x) Khỏi xin , hóng ờ tê lê trang mk',
  'image': 'https://sco